In [2]:
from keras.models import Sequential, clone_model, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, TimeDistributed, Conv2D, Reshape
from keras.layers import Convolution2D, MaxPooling2D, Concatenate, CuDNNLSTM, Cropping1D
from keras.losses import mean_squared_error
from keras.optimizers import Adam
from keras.utils import Sequence
import numpy as np
import tensorflow as tf
from keras import backend as K
tf.reset_default_graph()
TD = TimeDistributed
sess = tf.Session()
K.set_session(sess)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
batch_size = 32
learning_rate = 0.001
T_in = 16
T_total = 100
IMG_H = 125
IMG_W = 125
IMG_CH = 3

In [4]:
def build_autoencoder():
  inputs = Input(shape=(T_in, IMG_H, IMG_W, IMG_CH))
  # Apply convolutions on the initial image input with increasing channel size.
  conv_x = TD(Conv2D(32, kernel_size=(3, 3), strides=(2, 2), activation='relu'))(inputs)
  conv_x = TD(MaxPooling2D(pool_size=(2, 2)))(conv_x)
  conv_x = TD(Conv2D(64, kernel_size=(3, 3), strides=(2, 2), activation='relu'))(conv_x)
  conv_x = TD(MaxPooling2D(pool_size=(2, 2)))(conv_x)
  conv_x = TD(Conv2D(128, kernel_size=(3, 3), strides=(2, 2), activation='relu'))(conv_x)
  print("Exit initial conv: ", conv_x.shape)

  # We pass the flattened convolution output into a CuDNN-optimized LSTM.
  # Outputs are disregarded for training but form the "encoded" representation.
  enc_x = Reshape((T_in, -1))(conv_x)
  encoded = Concatenate()(CuDNNLSTM(1024, return_state=True, return_sequences=False)(enc_x))
  return Model(inputs=inputs, outputs=encoded)

autoencoder = build_autoencoder()
autoencoder.load_weights("/home/ubuntu/semisupervised_mil/autoencoder/model_weights_6.h5", by_name=True)

Exit initial conv:  (?, 16, 3, 3, 128)


In [5]:
def build_worse_model():
  inputs = Input(shape=(T_in, IMG_H, IMG_W, IMG_CH))
  model = TD(Flatten())(inputs)
  model = Cropping1D(cropping=(T_in - 1, 0))(model)
  model = Reshape((IMG_H, IMG_W, IMG_CH))(model)
  model = Convolution2D(32, (3, 3), padding='same', activation="relu")(model)
  model = MaxPooling2D(pool_size=(3, 3))(model)
  model = Convolution2D(32, (3, 3), padding='same', activation="relu")(model)
  model = MaxPooling2D(pool_size=(3, 3))(model)
  model = Dropout(0.25)(model)
  model = Flatten()(model)
  model = Dense(512, activation="relu")(model)
  model = Dropout(0.5)(model)
  preds = Dense(7, activation="tanh")(model)
  
  keras_model = Model(inputs=inputs, outputs=preds)
  
  return keras_model

In [6]:
def build_better_model():
  inputs = Input(shape=(T_in, IMG_H, IMG_W, IMG_CH))
  model = autoencoder(inputs)
  model = Dense(512, activation="relu")(model)
  model = Dropout(0.5)(model)
  preds = Dense(7, activation="tanh")(model)
  keras_model = Model(inputs=inputs, outputs=preds)
  
  return keras_model

In [7]:
# REMEMBER TO UPDATE
build_model = build_better_model

In [8]:
# Load train data
train_input = tf.placeholder(tf.float32, shape=(batch_size, T_in, IMG_H, IMG_W, IMG_CH))
train_label = tf.placeholder(tf.float32, shape=(batch_size, 7))
# Build initial model
before_model = build_model()
# Calulate initial loss
before_pred = before_model(train_input)
# Calculate loss and gradient for the task
before_loss = tf.reduce_mean(mean_squared_error(train_label, before_pred))
before_gradients = tf.gradients(before_loss, before_model.trainable_weights)
# Calculate ethereal weights for task-specific network
ethereal = {}
for weight, gradient in zip(before_model.trainable_weights, before_gradients):
  ethereal[weight] = weight - learning_rate * gradient
# Load test data
specialized_copy_ops = []
test_input = tf.placeholder(tf.float32, shape=(batch_size, T_in, IMG_H, IMG_W, IMG_CH))
test_label = tf.placeholder(tf.float32, shape=(batch_size, 7))
# Build new ethereal model
after_model = build_model()
for before_weight, after_weight in zip(before_model.trainable_weights,
                                       after_model.trainable_weights):
  specialized_copy_ops.append(tf.assign(after_weight, ethereal[before_weight]))
# Calculate the final gradients!
after_pred = after_model(test_input)
after_loss = tf.reduce_mean(mean_squared_error(test_label, after_pred))
after_grads = tf.gradients(after_loss, after_model.trainable_weights)

# Update our meta weights!
meta_ops = []
for before_weight, grad in zip(before_model.trainable_weights, after_grads):
  meta_ops.append(tf.assign(before_weight, before_weight - learning_rate * grad))

In [9]:
def train(train_x, train_y, test_x, test_y):
  sess.run(specialized_copy_ops, feed_dict={train_input: train_x,
                                            train_label: train_y})
  sess.run(meta_ops, feed_dict={test_input: test_x,
                                test_label: test_y})
  
def test(train_x, train_y, test_x, test_y):
  sess.run(specialized_copy_ops, feed_dict={train_input: train_x,
                                            train_label: train_y})
  return sess.run(after_loss, feed_dict={test_input: test_x,
                                         test_label: test_y})

In [10]:
# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [12]:
epochs = 1000
for i in range(epochs):
  for train_x, train_y, test_x, test_y in return_data(batch_size):
    for i in range(15, T_total):
      train(train_x[:,i-15:i+1], train_y[:,i], test_x[:,i-15:i+1], test_y[:,i])
      print(test(train_x[:,i-15:i+1], train_y[:,i], test_x[:,i-15:i+1], test_y[:,i]))

ValueError: too many values to unpack (expected 4)

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
show_graph(tf.get_default_graph().as_graph_def())

In [1]:
import pickle, random, os
import contextlib
@contextlib.contextmanager
def open_zip(filename, mode='r'):
    """
    Open a file; if filename ends with .gz, opens as a gzip file
    """
    if filename.endswith('.gz'):
        openfn = gzip.open
    else:
        openfn = open
    yield openfn(filename, mode)
class DataLogger(object):
    """
    This class pickles data into files and unpickles data from files.
    TODO: Handle logging text to terminal, GUI text, and/or log file at
        DEBUG, INFO, WARN, ERROR, FATAL levels.
    TODO: Handle logging data to terminal, GUI text/plots, and/or data
          files.
    """
    def __init__(self):
        pass

    def pickle(self, filename, data):
        """ Pickle data into file specified by filename. """
        with open_zip(filename, 'wb') as f:
            pickle.dump(data, f)

    def unpickle(self, filename):
        """ Unpickle data from file specified by filename. """
        try:
            with open_zip(filename, 'rb') as f:
                result = pickle.load(f, encoding='latin1')
            return result
        except IOError:
            LOGGER.debug('Unpickle error. Cannot find file: %s', filename)
            return None

def extract_demo_dict(demo_file):
    if type(demo_file) is not list:
        demos = DataLogger().unpickle(demo_file)
    else:
        demos = {}
        for i in range(0, len(demo_file)):
            demos[i] = DataLogger().unpickle(demo_file[i])
    return demos

from PIL import Image, ImageSequence
from natsort import natsorted
import glob
def gif_to_np(path):
  def __frame_to_np(frame):
    return np.array(frame.copy().convert("RGB").getdata(),
                    dtype=np.uint8).reshape(frame.size[1],
                                            frame.size[0], 3)

  im = Image.open(path)
  data = np.array(list(map(__frame_to_np, ImageSequence.Iterator(im))))
  return data


def load_path(paths_queue):
  for p in glob.glob(DATA_PATH + "/.gif"):
    paths_queue.put(p)
  return


def return_data(batch_size=5):
  """ 
    Returns one demoX entry, demoU entry, and corresponding gif entry
  """
  demo_file = '/home/ubuntu/data/sim_push/'
  demo_gif_dir = '/home/ubuntu/data/sim_push/'
  gif_prefix = 'object'

  demo_files = natsorted(glob.glob(demo_file + '/*pkl'))
  result1 = {}
  result2 = {}
  for num in range(0, len(demo_files), batch_size):
    batch_result = ()
    for b in range(num, batch_size):

      # Extracting data from the applicable demo
      demos = extract_demo_dict([demo_files[b]])
      key = list(demos.keys())[0]

      # Numbers for the two random samples being picked
      rand_entry_1, rand_entry_2 = random.sample(range(0, len(demos[key]['demoX'])), 2)

      gif_folder = os.path.join(demo_gif_dir, gif_prefix + '_%d' % b)
      image_paths = natsorted(os.listdir(gif_folder))

      #result1['demoX'] = demos[key]['demoX'][rand_entry_1]
      result1['demoU'] = demos[key]['demoU'][rand_entry_1]
      result1['gif'] = gif_to_np(gif_folder + '/' +  image_paths[rand_entry_1])

      #result2['demoX'] = demos[key]['demoX'][rand_entry_2]
      result2['demoU'] = demos[key]['demoU'][rand_entry_2]
      result2['gif'] = gif_to_np(gif_folder + '/' +  image_paths[rand_entry_2])

      batch_result += result1['demoU'], result1['gif'], result2['demoU'], result2['gif']
    yield batch_result